In [35]:
import pandas as pd
import numpy as np
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
#import columns to be dropped from data discovery 

columns_to_drop =['member_id', 'desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'verification_status_joint', 'mths_since_last_delinq','batch_enrolled' ]

In [3]:
data= pd.read_csv('train_indessa.csv')

In [4]:
#drop the columns
#dropping title as well, because of less time , not running sentiment analysis
data.drop(['member_id','desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'verification_status_joint', 
           'mths_since_last_delinq','batch_enrolled','title' ],inplace=True,axis=1 )

In [5]:
#define features 
features = data.columns
print(features, features.shape)

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'pymnt_plan', 'purpose',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med',
       'application_type', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'loan_status'],
      dtype='object') (37,)


In [6]:
#lets preprocess categorical columns first 
var_cat = []
for var in features:
    if data[var].dtypes == 'O':
        var_cat.append(var)
print(var_cat)

['term', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'zip_code', 'addr_state', 'initial_list_status', 'application_type', 'last_week_pay']


In [7]:
(data[var_cat].isnull().mean()*100)

term                   0.000000
grade                  0.000000
sub_grade              0.000000
emp_title              5.791018
emp_length             5.050636
home_ownership         0.000000
verification_status    0.000000
pymnt_plan             0.000000
purpose                0.000000
zip_code               0.000000
addr_state             0.000000
initial_list_status    0.000000
application_type       0.000000
last_week_pay          0.000000
dtype: float64

In [11]:
random_impute= ['emp_length','emp_title']

In [12]:
#single go pandas is going heavy on data , so running it in two chunks 
def random_imputatiochunk1(data, column):
    
    n=len(data)//2
    for x in range(0,n):
        if pd.isna(data[column][x]):
            #print(data['emp_length'][x])
            data[column][x]=np.random.choice(data[column].dropna().values)
            
def random_imputatiochunk2(data, column):
    
    n=len(data)//2
    for x in range(n,532428):
        if pd.isna(data[column][x]):
            #print(data['emp_length'][x])
            data[column][x]=np.random.choice(data[column].dropna().values)

In [13]:
for i in random_impute:
    random_imputatiochunk1(data,i)
    random_imputatiochunk2(data,i)

<ipython-input-12-28dd46761017>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column][x]=np.random.choice(data[column].dropna().values)
<ipython-input-12-28dd46761017>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column][x]=np.random.choice(data[column].dropna().values)


In [16]:
#process Column with str+int
def to_int(data,column):
    data[column+'_int'] = data[column].str.extract('(\d+)').astype(int) 

In [25]:
to_int_columns= ['term','sub_grade','last_week_pay']

In [29]:
# last_week_pay has value NAth week , which needs to be converted into int+str
#this values can be handled by random imputation , for now replaced by 0
data['last_week_pay'].replace({'NAth week':'0th week'}, inplace=True)

In [31]:
for i in to_int_columns:
    print(i)
    to_int(data,i)

term
sub_grade
last_week_pay


In [33]:
#lable encode emp length 
coding_dict= {'< 1 year':0, '1 year':1, '2 years':2, '9 years':9, '10+ years':10, '5 years':5,
       '8 years':8, '7 years':7, '4 years':4, '3 years':3, '6 years':6  }
data['emp_length']= data['emp_length'].map(coding_dict)

In [84]:
#columns to be lable encoded 
tobe_lable_encode= ['grade ', 'home_ownership ', 'verification_status ', 'pymnt_plan ', 
                    'purpose ',  'addr_state ', 'initial_list_status',  'application_type ' ]

In [115]:
#use sklearn lable encoder 
grade= LabelEncoder()
data['grade']= grade.fit_transform(data['grade'])

home_ownership= LabelEncoder()
data['home_ownership']= home_ownership.fit_transform(data['home_ownership'])
    
verification_status= LabelEncoder()
data['verification_status']= verification_status.fit_transform(data['verification_status'])

pymnt_plan= LabelEncoder()
data['pymnt_plan']= pymnt_plan.fit_transform(data['pymnt_plan'])

purpose= LabelEncoder()
data['purpose']= purpose.fit_transform(data['purpose'])

addr_state= LabelEncoder()
data['addr_state']= addr_state.fit_transform(data['addr_state'])

initial_list_status= LabelEncoder()
data['initial_list_status']= initial_list_status.fit_transform(data['initial_list_status'])

application_type= LabelEncoder()
data['application_type']= application_type.fit_transform(data['application_type'])

In [89]:
#mean target encoding 

pur= data.groupby('purpose')['loan_status'].mean()
data['purpose']= data['purpose'].map(pur)

In [85]:
data.drop(['sub_grade','term','last_week_pay'], axis=1, inplace=True)

In [91]:
#coun frequency encoding for emp_title

emp_ttl= data.groupby('emp_title')['loan_status'].count()

In [95]:
data['emp_title']= data['emp_title'].map(emp_ttl)

In [105]:
##preprocessign Numerical data 


In [100]:
## as per correlation matrix , we can delete some columns
columns_to_drop = ['loan_amnt', 'funded_amnt','total_rev_hi_lim','open_acc','collection_recovery_fee']

In [101]:
data.drop(['loan_amnt', 'funded_amnt','total_rev_hi_lim','open_acc','collection_recovery_fee' ],inplace=True,axis=1 )

In [106]:
features_n= data.columns
var_num = [] 
for var in features_n: 
    if data[var].dtypes != 'O': 
        var_num.append(var) 
print(var_num)

['funded_amnt_inv', 'int_rate', 'grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'loan_status', 'term_int', 'sub_grade_int', 'last_week_pay_int']


In [103]:
from sklearn.impute import SimpleImputer

In [107]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data[var_num])
x=imp.transform(data[var_num])
data[var_num]= pd.DataFrame(x)

In [109]:
data.isna().sum()

funded_amnt_inv               0
int_rate                      0
grade                         0
emp_title                     0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
pymnt_plan                    0
purpose                       0
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
total_rec_int                 0
total_rec_late_fee            0
recoveries                    0
collections_12_mths_ex_med    0
application_type              0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
loan_status                   0
term_int                      0
sub_grade_int                 0
last_wee

In [116]:
data.head()
#data['term_int'] = data['term'].str.extract('(\d+)').astype(int)

,funded_amnt_inv,int_rate,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,...,recoveries,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,loan_status,term_int,sub_grade_int,last_week_pay_int
0,14350.0,19.19,4,319.0,9.0,4,28700.0,1,0,1,...,0.0,0.0,0,0.0,0.0,28699.0,0.0,36.0,3.0,26.0
1,4800.0,10.99,1,76.0,0.0,1,65000.0,1,0,2,...,0.0,0.0,0,0.0,0.0,9974.0,0.0,36.0,4.0,9.0
2,10000.0,7.26,0,2309.0,2.0,4,45000.0,0,0,1,...,0.0,0.0,0,0.0,65.0,38295.0,0.0,36.0,4.0,9.0
3,15000.0,19.72,3,1.0,10.0,5,105000.0,0,0,1,...,0.0,0.0,0,0.0,0.0,55564.0,0.0,36.0,5.0,135.0
4,16000.0,10.64,1,1.0,10.0,5,52000.0,2,0,0,...,0.0,0.0,0,0.0,0.0,47159.0,0.0,36.0,2.0,96.0


In [117]:
data.to_csv('processed_train.csv', encoding='utf-8',index=False)